In [1]:
import numpy as np
import pandas as pd
import torch
from torch import nn
from torch.utils.data import DataLoader, TensorDataset
import adabound

In [2]:
training_data = pd.read_csv("data/train.csv")

In [3]:
training_data.fillna(0, inplace=True)

In [4]:
seperated_columns = training_data['Cabin'].str.split('/', expand=True)
training_data[['C1', 'C2', 'C3']] = seperated_columns

In [5]:
seperated_columns = training_data['PassengerId'].str.split('_', expand=True)
training_data[['PI1', 'PI2']] = seperated_columns.astype('int')

In [6]:
seperated_columns = training_data['Name'].str.split(' ', expand=True)
training_data['Last_name'] = seperated_columns[1]
full_training_data = training_data
training_data = full_training_data.drop(columns=['PassengerId', 'Cabin', 'Name'])

In [7]:
cols = training_data.columns.tolist()
print(cols)
cols = cols[:10] + cols[11:] + [cols[10]]
training_data = training_data[cols]
training_data.fillna(0, inplace=True)
training_data

['HomePlanet', 'CryoSleep', 'Destination', 'Age', 'VIP', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck', 'Transported', 'C1', 'C2', 'C3', 'PI1', 'PI2', 'Last_name']


,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,C1,C2,C3,PI1,PI2,Last_name,Transported
0,Europa,False,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,B,0,P,1,1,Ofracculy,False
1,Earth,False,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,F,0,S,2,1,Vines,True
2,Europa,False,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,A,0,S,3,1,Susent,False
3,Europa,False,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,A,0,S,3,2,Susent,False
4,Earth,False,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,F,1,S,4,1,Santantines,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,Europa,False,55 Cancri e,41.0,True,0.0,6819.0,0.0,1643.0,74.0,A,98,P,9276,1,Noxnuther,False
8689,Earth,True,PSO J318.5-22,18.0,False,0.0,0.0,0.0,0.0,0.0,G,1499,S,9278,1,Mondalley,False
8690,Earth,False,TRAPPIST-1e,26.0,False,0.0,0.0,1872.0,1.0,0.0,G,1500,S,9279,1,Connon,True
8691,Europa,False,55 Cancri e,32.0,False,0.0,1049.0,0.0,353.0,3235.0,E,608,S,9280,1,Hontichre,False


In [8]:
training_data = training_data.to_numpy()
training_data

array([['Europa', False, 'TRAPPIST-1e', ..., 1, 'Ofracculy', False],
       ['Earth', False, 'TRAPPIST-1e', ..., 1, 'Vines', True],
       ['Europa', False, 'TRAPPIST-1e', ..., 1, 'Susent', False],
       ...,
       ['Earth', False, 'TRAPPIST-1e', ..., 1, 'Connon', True],
       ['Europa', False, '55 Cancri e', ..., 1, 'Hontichre', False],
       ['Europa', False, 'TRAPPIST-1e', ..., 2, 'Hontichre', True]],
      dtype=object)

In [9]:
def class_mapping(column_number):
    classes = {}
    index = 0
    for id in training_data[:, column_number]:
        if id not in classes:
            classes[id] = index
            index += 1
    return classes


def class_mapping_for_name(column_number):
    classes = {0: 0}
    index = 1
    for id in training_data[:, column_number]:
        if id not in classes:
            classes[id] = index
            index += 1
    return classes

In [10]:
planet_map = class_mapping(0)

In [11]:
TF_map = class_mapping(1)
TF_map

{False: 0, True: 1}

In [12]:
destination_map = class_mapping(2)
destination_map

{'TRAPPIST-1e': 0, 'PSO J318.5-22': 1, '55 Cancri e': 2, 0: 3}

In [13]:
cabin0 = class_mapping(10)
cabin0

{'B': 0, 'F': 1, 'A': 2, 'G': 3, 0: 4, 'E': 5, 'D': 6, 'C': 7, 'T': 8}

In [14]:
cabin2 = class_mapping(12)
cabin2

{'P': 0, 'S': 1, 0: 2}

In [15]:
last_name_map = class_mapping_for_name(15)
print(len(last_name_map))
last_name_map

2218


{0: 0,
 'Ofracculy': 1,
 'Vines': 2,
 'Susent': 3,
 'Santantines': 4,
 'Hinetthews': 5,
 'Jacostaffey': 6,
 'Beston': 7,
 'Flatic': 8,
 'Barne': 9,
 'Baketton': 10,
 'Bertsontry': 11,
 'Pooles': 12,
 'Eccle': 13,
 'Hughriend': 14,
 'Upead': 15,
 'Brighttt': 16,
 'Brantuarez': 17,
 'Mcfaddennon': 18,
 'Jacostanley': 19,
 'Fullided': 20,
 'Brookenson': 21,
 'Unconary': 22,
 'Mare': 23,
 'Morsentley': 24,
 'Datie': 25,
 'Oingwhedly': 26,
 'Butte': 27,
 'Leodger': 28,
 'Wheelez': 29,
 'Batthewitt': 30,
 'Moodsey': 31,
 'Cylistrand': 32,
 'Coopelandez': 33,
 'Chmad': 34,
 'Lancis': 35,
 'Johnshines': 36,
 'Hubbarton': 37,
 'Hickerson': 38,
 'Tractive': 39,
 'Ayalazquez': 40,
 'Salez': 41,
 'Greeves': 42,
 'Keen': 43,
 'Pecketton': 44,
 'Leeves': 45,
 'Binie': 46,
 'Dillines': 47,
 'Make': 48,
 'Mostedry': 49,
 'Dal': 50,
 'Blité': 51,
 'Pokerheed': 52,
 'Vinozarks': 53,
 'Datte': 54,
 'Coopezmaney': 55,
 'Ellcefulve': 56,
 'Kinson': 57,
 'Sacre': 58,
 'Santry': 59,
 'Yorkland': 60,
 'Dunnis

In [16]:
def replace_with_mapped_value(column_number, map_name, data_set):
    data_set[:, column_number] = np.vectorize(map_name.get)(data_set[:, column_number])

In [17]:
replace_with_mapped_value(0, planet_map, training_data)
training_data

array([[0, False, 'TRAPPIST-1e', ..., 1, 'Ofracculy', False],
       [1, False, 'TRAPPIST-1e', ..., 1, 'Vines', True],
       [0, False, 'TRAPPIST-1e', ..., 1, 'Susent', False],
       ...,
       [1, False, 'TRAPPIST-1e', ..., 1, 'Connon', True],
       [0, False, '55 Cancri e', ..., 1, 'Hontichre', False],
       [0, False, 'TRAPPIST-1e', ..., 2, 'Hontichre', True]], dtype=object)

In [18]:
replace_with_mapped_value(1, TF_map, training_data)
replace_with_mapped_value(2, destination_map, training_data)
replace_with_mapped_value(4, TF_map, training_data)
replace_with_mapped_value(10, cabin0, training_data)
replace_with_mapped_value(12, cabin2, training_data)
replace_with_mapped_value(15, last_name_map, training_data)
replace_with_mapped_value(16, TF_map, training_data)

In [19]:
training_data

array([[0, 0, 0, ..., 1, 1, 0],
       [1, 0, 0, ..., 1, 2, 1],
       [0, 0, 0, ..., 1, 3, 0],
       ...,
       [1, 0, 0, ..., 1, 225, 1],
       [0, 0, 2, ..., 1, 380, 0],
       [0, 0, 0, ..., 2, 380, 1]], dtype=object)

In [20]:
np.random.shuffle(training_data)
training_data, test_data = training_data[:int(len(training_data) * 0.8)], training_data[int(len(training_data) * 0.8):]
# np.random.shuffle(training_data)
# training_data, test_data = training_data, training_data


# x=np.delete(training_data,[0,1,2],1).astype(float)
# y=training_data[:,[1]].astype(float)
# tensor_x=torch.Tensor(x)
# tensor_y=torch.Tensor(y)
# training_data=TensorDataset(tensor_x,tensor_y)
def to_tensor(np_array, target_column):
    x = np.delete(np_array, target_column, axis=1).astype(float)
    y = np_array[:, target_column].astype(int)
    y_with_zeros = []
    for class_element in y:
        y_with_zeros.append(np.zeros(len(TF_map)))
        y_with_zeros[-1][class_element] = 1
    y = y_with_zeros
    tensor_x = torch.Tensor(x)
    tensor_y = torch.Tensor(y)
    tensor = TensorDataset(tensor_x, tensor_y)
    return tensor

In [21]:
batch_size = 128

# Create data loaders.
train_dataloader = DataLoader(to_tensor(training_data, 16), batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(to_tensor(test_data, 16), batch_size=batch_size)

for X, y in train_dataloader:
    print(X.shape)
    print(y.shape, y.dtype)
    break

# # Display sample data
# figure = plt.figure(figsize=(10, 8))
# cols, rows = 5, 5
# for i in range(1, cols * rows + 1):
#     idx = torch.randint(len(test_data), size=(1,)).item()
#     img, label = test_data[idx]
#     figure.add_subplot(rows, cols, i)
#     plt.title(label)
#     plt.axis("off")
#     plt.imshow(img.squeeze(), cmap="gray")
# plt.show()

torch.Size([128, 16])
torch.Size([128, 2]) torch.float32


C:\Users\shaow\AppData\Local\Temp\ipykernel_22812\1188960576.py:21: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  C:\cb\pytorch_1000000000000\work\torch\csrc\utils\tensor_new.cpp:210.)
  tensor_y = torch.Tensor(y)


In [22]:
# Get cpu or gpu device for training.
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using {} device".format(device))
layer_size = 512
input_size = 16


# Define model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(input_size, layer_size),
            nn.ReLU(),
            nn.Linear(layer_size, layer_size),
            nn.ReLU(),
            nn.Linear(layer_size, layer_size),
            nn.ReLU(),
            nn.Linear(layer_size, layer_size),
            nn.ReLU(),
            nn.Linear(layer_size, layer_size),
            nn.ReLU(),
            nn.Linear(layer_size, layer_size),
            nn.ReLU(),
            nn.Linear(layer_size, layer_size),
            nn.ReLU(),
            nn.Linear(layer_size, layer_size),
            nn.ReLU(),
            nn.Linear(layer_size, layer_size),
            nn.ReLU(),
            nn.Linear(layer_size, layer_size),
            nn.ReLU(),
            nn.Linear(layer_size, layer_size),
            nn.ReLU(),
            nn.Linear(layer_size, layer_size),
            nn.ReLU(),
            nn.Linear(layer_size, layer_size),
            nn.ReLU(),
            nn.Linear(layer_size, layer_size),
            nn.ReLU(),
            nn.Linear(layer_size, layer_size),
            nn.ReLU(),
            nn.Linear(layer_size, layer_size),
            nn.ReLU(),
            nn.Linear(layer_size, layer_size),
            nn.ReLU(),
            nn.Linear(layer_size, len(TF_map)),
            nn.Sigmoid()
        )

    def forward(self, x):
        logits = self.linear_relu_stack(x)
        return logits


model = NeuralNetwork().to(device)
print(model)

Using cuda device
NeuralNetwork(
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=16, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=512, bias=True)
    (5): ReLU()
    (6): Linear(in_features=512, out_features=512, bias=True)
    (7): ReLU()
    (8): Linear(in_features=512, out_features=512, bias=True)
    (9): ReLU()
    (10): Linear(in_features=512, out_features=512, bias=True)
    (11): ReLU()
    (12): Linear(in_features=512, out_features=512, bias=True)
    (13): ReLU()
    (14): Linear(in_features=512, out_features=512, bias=True)
    (15): ReLU()
    (16): Linear(in_features=512, out_features=512, bias=True)
    (17): ReLU()
    (18): Linear(in_features=512, out_features=512, bias=True)
    (19): ReLU()
    (20): Linear(in_features=512, out_features=512, bias=True)
    (21): ReLU()
    (22): Linear(in_features=512, out_features=512, bias=True)


In [23]:
# loss_fn = nn.L1Loss()
loss_fn = nn.MSELoss()
learning_rate = 1e-3
# optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
optimizer = adabound.AdaBound(model.parameters(), lr=learning_rate,final_lr=0.1)

In [24]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        # y=y.type(torch.float)
        X, y = X.to(device), y.to(device)
        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 8 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


In [25]:
def test(dataloader, model):
    size = len(dataloader.dataset)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y.argmax(1)).type(torch.float).sum().item()
    test_loss /= size
    correct /= size
    try:
        if correct > 0.82:
            torch.save(model, 'model/m3')
        elif correct > 0.815:
            torch.save(model, 'model/m2')
        elif correct > 0.81:
            torch.save(model, 'model/m1')
        elif correct > 0.805:
            torch.save(model, 'model/m0')
        elif correct > 0.8:
            torch.save(model, 'model/ml')
    except Exception:
        pass

    print(f"Test Error: \n Accuracy: {(100 * correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [26]:
epochs = 1000
for t in range(epochs):
    print(f"Epoch {t + 1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model)
print("Done!")

Epoch 1
-------------------------------


C:\Users\shaow\anaconda3\envs\study\lib\site-packages\adabound\adabound.py:94: UserWarning: This overload of add_ is deprecated:
	add_(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add_(Tensor other, *, Number alpha) (Triggered internally at  C:\cb\pytorch_1000000000000\work\torch\csrc\utils\python_arg_parser.cpp:1055.)
  exp_avg.mul_(beta1).add_(1 - beta1, grad)


loss: 0.250408  [    0/ 6954]
loss: 0.250017  [ 1024/ 6954]
loss: 0.249839  [ 2048/ 6954]
loss: 0.250481  [ 3072/ 6954]
loss: 0.250053  [ 4096/ 6954]
loss: 0.250172  [ 5120/ 6954]
loss: 0.249850  [ 6144/ 6954]
Test Error: 
 Accuracy: 48.2%, Avg loss: 0.002013 

Epoch 2
-------------------------------
loss: 0.250150  [    0/ 6954]
loss: 0.249652  [ 1024/ 6954]
loss: 0.249956  [ 2048/ 6954]
loss: 0.249995  [ 3072/ 6954]
loss: 0.249445  [ 4096/ 6954]
loss: 0.249863  [ 5120/ 6954]
loss: 0.250075  [ 6144/ 6954]
Test Error: 
 Accuracy: 48.2%, Avg loss: 0.002013 

Epoch 3
-------------------------------
loss: 0.249438  [    0/ 6954]
loss: 0.250900  [ 1024/ 6954]
loss: 0.249774  [ 2048/ 6954]
loss: 0.249893  [ 3072/ 6954]
loss: 0.249653  [ 4096/ 6954]
loss: 0.250026  [ 5120/ 6954]
loss: 0.249593  [ 6144/ 6954]
Test Error: 
 Accuracy: 73.5%, Avg loss: 0.002007 

Epoch 4
-------------------------------
loss: 0.249376  [    0/ 6954]
loss: 0.249074  [ 1024/ 6954]
loss: 0.248978  [ 2048/ 6954]
loss

KeyboardInterrupt: 

In [ ]:
training_data = pd.read_csv("data/test.csv")
training_data.fillna(0, inplace=True)
seperated_columns = training_data['Cabin'].str.split('/', expand=True)
training_data[['C1', 'C2', 'C3']] = seperated_columns
seperated_columns = training_data['PassengerId'].str.split('_', expand=True)
training_data[['PI1', 'PI2']] = seperated_columns.astype('int')
seperated_columns = training_data['Name'].str.split(' ', expand=True)
training_data['Last_name'] = seperated_columns[1]
full_training_data = training_data
training_data = full_training_data.drop(columns=['Cabin', 'Name'])
cols = training_data.columns.tolist()
print(cols)
cols = cols[1:] + cols[:1]
print(cols)
training_data = training_data[cols]
training_data.fillna(0, inplace=True)
training_data = training_data.to_numpy()
training_data

In [ ]:
def replace_with_mapped_value_for_names(column_number, map_name, data_set):
    def name_only(key):
        value = last_name_map.get(key)
        if value is None:
            return 0
        else:
            return value

    data_set[:, column_number] = np.vectorize(name_only)(data_set[:, column_number])

In [ ]:
replace_with_mapped_value(0, planet_map, training_data)
replace_with_mapped_value(1, TF_map, training_data)
replace_with_mapped_value(2, destination_map, training_data)
replace_with_mapped_value(4, TF_map, training_data)
replace_with_mapped_value(10, cabin0, training_data)
replace_with_mapped_value(12, cabin2, training_data)
replace_with_mapped_value_for_names(15, last_name_map, training_data)
training_data

In [ ]:
model = torch.load('model/ml')

In [ ]:
dct = {v: k for k, v in TF_map.items()}
dct

In [ ]:
result = []
for line in training_data:
    current_line = torch.Tensor(line[:16].astype(float)).to(device)
    pred = model(current_line)
    result.append([line[16], dct[pred.argmax().item()]])
result

In [ ]:
result = [['PassengerId', 'Transported']] + result
result

In [ ]:
np.savetxt("result.csv", result, delimiter=",", fmt='%s')